In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI: Train and deploy LightGBM models with custom container

### Dataset

The dataset used for this tutorial is the [Iris dataset](https://www.tensorflow.org/datasets/catalog/iris) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview). This dataset does not require any feature engineering. The version of the dataset you will use in this tutorial is stored in a public Cloud Storage bucket. The trained model predicts the type of Iris flower species from a class of three species: setosa, virginica, or versicolor.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

If you are using Colab or Google Cloud Notebooks, your environment already meets all the requirements to run this notebook. You can skip this step.

Otherwise, make sure your environment meets this notebook's requirements. You need the following:

- The Cloud Storage SDK
- Git
- Python 3
- virtualenv
- Jupyter notebook running in a virtual environment with Python 3

The Cloud Storage guide to [Setting up a Python development environment](https://cloud.google.com/python/setup) and the [Jupyter installation guide](https://jupyter.org/install) provide detailed instructions for meeting these requirements. The following steps provide a condensed set of instructions:

1. [Install and initialize the SDK](https://cloud.google.com/sdk/docs/).

2. [Install Python 3](https://cloud.google.com/python/setup#installing_python).

3. [Install virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) and create a virtual environment that uses Python 3.  Activate the virtual environment.

4. To install Jupyter, run `pip3 install jupyter` on the command-line in a terminal shell.

5. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

6. Open this notebook in the Jupyter Notebook Dashboard.


## Installation

Install the latest version of Vertex SDK for Python.

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

Install the latest GA version of *google-cloud-storage* library as well.

In [ ]:
!pip3 install -U google-cloud-storage $USER_FLAG

Install latest version of [`ligthgbm`](https://pypi.org/project/lightgbm/) package

In [ ]:
!pip3 install -U lightgbm $USER_FLAG
#!conda install -c conda-forge -y lightgbm

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU runtime

This tutorial does not require a GPU runtime.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the following APIs: Vertex AI APIs, Compute Engine APIs, and Cloud Storage.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. If you are running this notebook locally, you will need to install the [Cloud SDK]((https://cloud.google.com/sdk)).

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

In the Cloud Console, go to the [Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey) page.

**Click Create service account**.

In the **Service account name** field, enter a name, and click **Create**.

In the **Grant this service account access to project** section, click the Role drop-down list. Type "Vertex" into the filter box, and select **Vertex Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

Click Create. A JSON file that contains your key downloads to your local environment.

Enter the path to your service account key as the GOOGLE_APPLICATION_CREDENTIALS variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [ ]:
import os

import google.cloud.aiplatform as aiplatform
import lightgbm

In [ ]:
print(f"LightGBM version {lightgbm.__version__}")

In [ ]:
APP_NAME = "iris"

## Initialize Vertex SDK for Python

Initialize the Vertex SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

#### Set pre-built containers

Set the pre-built Docker container image for training and prediction.


For the latest list, see [Pre-built containers for training](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers).


For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [ ]:
TRAIN_VERSION = "lightgbm-cpu"
DEPLOY_VERSION = "lightgbm-cpu"

TRAIN_IMAGE = f"gcr.io/{PROJECT_ID}/training/{TRAIN_VERSION}:latest"
DEPLOY_IMAGE = f"gcr.io/{PROJECT_ID}/prediction/{DEPLOY_VERSION}:latest"

#### Set machine type

Next, set the machine type to use for training and prediction.

- Set the variables `TRAIN_COMPUTE` and `DEPLOY_COMPUTE` to configure  the compute resources for the VMs you will use for for training and prediction.
 - `machine type`
     - `n1-standard`: 3.75GB of memory per vCPU.
     - `n1-highmem`: 6.5GB of memory per vCPU
     - `n1-highcpu`: 0.9 GB of memory per vCPU
 - `vCPUs`: number of \[2, 4, 8, 16, 32, 64, 96 \]

*Note: The following is not supported for training:*

 - `standard`: 2 vCPUs
 - `highcpu`: 2, 4 and 8 vCPUs

*Note: You may also use n2 and e2 machine types for training and deployment, but they do not support GPUs*.

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

### Examine the training package

![training-with-custom-containers-on-vertex-training](./images/training-with-custom-containers-on-vertex-training.png)

#### Package layout

Before you start the training, you will look at how a custom container is created for a custom training job. The training application code is contains the following directory/file layout.

- trainer
  - \_\_init\_\_.py
  - task.py

The file `trainer/task.py` is the Python script for executing the custom training job. *Note*, when we referred to it in the worker pool specification, we replace the directory slash with a dot (`trainer.task`) and dropped the file suffix (`.py`).

#### Building custom training container

In the following cells, you will build a custom training container.

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

In [ ]:
%%writefile custom/trainer/task.py
# Single Instance Training for Iris

import datetime
import os
import subprocess
import sys

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import pandas as pd

import lightgbm as lgb

import argparse

print("Parsing arguments")

parser = argparse.ArgumentParser()
parser.add_argument(
    '--model-dir', 
    dest='model_dir',        
    default=os.getenv('AIP_MODEL_DIR'), 
    type=str, 
    help='Location to export GCS model')
args = parser.parse_args()
print(args)

# Download data
print("Downloading data")
iris = load_iris()
print(iris.data.shape)

# split data
print("Splitting data into test and train")
x, y = iris.data, iris.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

# create dataset for lightgbm
print("creating dataset for LightGBM")
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'metric': {'multi_error'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'num_class' : 3
}

# train lightgbm model
print('Starting training...')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

# save model to file
print('Saving model...')
model_filename = 'model.txt'
gbm.save_model(model_filename)

# Upload the saved model file to Cloud Storage
gcs_model_path = os.path.join(args.model_dir, model_filename)
print(f"Copying model files to {gcs_model_path}")
subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path],
    stderr=sys.stdout)

#### Build and push custom container with training script

Next, you package the training folder into a container, and then push it to Container Registry or Artifact Registry

In [ ]:
%%writefile ./custom/Dockerfile

FROM gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23

WORKDIR /trainer
RUN pip3 install -U lightgbm

# Copies the trainer code to the docker image.
COPY ./trainer/ ./trainer/

ENTRYPOINT ["python3", "-m", "trainer.task"]

In [ ]:
!docker build --tag {TRAIN_IMAGE} ./custom

In [ ]:
!docker run -it --rm $TRAIN_IMAGE

In [ ]:
!docker push {TRAIN_IMAGE}

## Train a model

### [Training with custom containers](https://cloud.google.com/vertex-ai/docs/training/containers-overview)

### Create and run custom training job


To train a custom model, you perform two steps: 1) create a custom training job, and 2) run the job.

#### Create custom training job

A custom training job is created with the `CustomContainerTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the custom training job.
- `container_uri`: The custom training container image.

In [ ]:
JOB_NAME = f"{APP_NAME}-lightgbm-cstm-cntr-{TIMESTAMP}"

print(f"APP_NAME={APP_NAME}")
print(f"CUSTOM_TRAIN_IMAGE_URI={TRAIN_IMAGE}")
print(f"JOB_NAME={JOB_NAME}")

In [ ]:
# configure the job with container image spec
job = aiplatform.CustomContainerTrainingJob(
    display_name=f"{JOB_NAME}", 
    container_uri=f"{TRAIN_IMAGE}"
)

In [ ]:
print(vars(job))

*Example output:*

    <google.cloud.aiplatform.training_jobs.CustomTrainingJob object at 0x7feab1346710>

#### Run the custom training job

Next, you run the custom job to start the training job by invoking the method `run`, with the following parameters:

- `replica_count`: The number of compute instances for training (replica_count = 1 is single node training).
- `machine_type`: The machine type for the compute instances.
- `base_output_dir`: The Cloud Storage location to write the model artifacts to.
- `sync`: Whether to block until completion of the job.

In [ ]:
MODEL_DIR = f"{BUCKET_NAME}/{APP_NAME}-lightgbm-cstm-cntr/{TIMESTAMP}"
print(f"Base output directory {MODEL_DIR}/")

model = job.run(
    replica_count=1, 
    machine_type=TRAIN_COMPUTE, 
    base_output_dir=f"{MODEL_DIR}/", 
    sync=True
)

In [ ]:
model_path_to_deploy = MODEL_DIR + '/model/'
print(f"Model path with trained model artifacts {model_path_to_deploy}")

In [ ]:
!gsutil ls -r $model_path_to_deploy

## Deploy LightGBM model to Vertex AI

![serving-with-custom-containers-on-vertex-predictions](./images/serving-with-custom-containers-on-vertex-predictions.png)

### Build a HTTP Server with FastAPI

Custom Container image [requires](https://cloud.google.com/ai-platform-unified/docs/predictions/custom-container-requirements#image) that the container must run an HTTP server. Specifically, the container must listen and respond to liveness checks, health checks, and prediction requests.

In this tutorial, we will use FastAPI to implement the HTTP server. The HTTP server must listen for requests on 0.0.0.0.

In [ ]:
# Make folder for Python training script
! rm -rf serve
! mkdir serve

# Make the predictor subfolder
! mkdir serve/app

In [ ]:
%%writefile serve/app/main.py
from fastapi.logger import logger
from fastapi import FastAPI, Request

import numpy as np
import os

from sklearn.datasets import load_iris
import lightgbm as lgb

import logging

gunicorn_logger = logging.getLogger('gunicorn.error')
logger.handlers = gunicorn_logger.handlers

if __name__ != "main":
    logger.setLevel(gunicorn_logger.level)
else:
    logger.setLevel(logging.DEBUG)

app = FastAPI()

model_f = "/model/model.txt"

logger.info("Loading model")
_model = lgb.Booster(model_file=model_f)
logger.info("Loading target class labels")
_class_names = load_iris().target_names

@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    """ health check to ensure HTTP server is ready to handle 
        prediction requests
    """
    return {"status": "healthy"}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()

    instances = body["instances"]
    inputs = np.asarray(instances)
    
    outputs = _model.predict(inputs)

    logger.info(f"Outputs {outputs}")
    return {"predictions": [_class_names[class_num] for class_num in np.argmax(outputs, axis=1)]}

In [ ]:
%%writefile serve/requirements.txt

numpy~=1.20
scikit-learn~=0.24
pandas==1.0.4
lightgbm==3.2.1
google-cloud-storage>=1.26.0,<2.0.0dev

### Add pre-start script
FastAPI will execute this script before starting up the server. The `PORT` environment variable is set to equal `AIP_HTTP_PORT` in order to run FastAPI on same the port expected by AI Platform (Unified).

In [ ]:
%%writefile serve/app/prestart.sh

#!/bin/bash
export PORT=$AIP_HTTP_PORT

### Store test instances to use later
To learn more about formatting input instances in JSON, [read the documentation.](https://cloud.google.com/ai-platform-unified/docs/predictions/online-predictions-custom-models#request-body-details)

In [ ]:
%%writefile serve/instances.json
{
    "instances": [
        [6.7, 3.1, 4.7, 1.5],
        [4.6, 3.1, 1.5, 0.2]
    ]
}

## Build and push prediction container to Container Registry

Write the Dockerfile, using `tiangolo/uvicorn-gunicorn-fastapi` as a base image. This will automatically run FastAPI for you using Gunicorn and Uvicorn. Visit [the FastAPI docs to read more about deploying FastAPI with Docker](https://fastapi.tiangolo.com/deployment/docker/).

In [ ]:
%%bash -s $MODEL_DIR 

MODEL_DIR=$1

mkdir -p ./serve/model/
gsutil cp -r ${MODEL_DIR}/model/ ./serve/model/ 

cat > ./serve/Dockerfile <<EOF
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.7

COPY ./app /app
COPY ./model /
COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

EXPOSE 7080
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "7080"]

EOF

In [ ]:
AIP_STORAGE_URI=f"{MODEL_DIR}"
print(AIP_STORAGE_URI)

*Build* the image and tag the Artifact Registry path that you will push to.

In [ ]:
!docker build --tag={DEPLOY_IMAGE} ./serve

### Run and test the container locally (optional)

Run the container locally in detached mode and provide the environment variables that the container requires. These env vars will be provided to the container by AI Platform Prediction once deployed. Test the `/health` and `/predict` routes, then stop the running image.

Before push the container image to Container Registry to use it with Vertex Predictions, you can run it as a container in your local environment to verify that the server works as expected

- To run the container image as a container locally, run the following command:

In [ ]:
!docker rm local-iris
!docker run -t -d --rm -p 7080:7080 \
    --name=local-iris \
    -e AIP_HTTP_PORT=7080 \
    -e AIP_HEALTH_ROUTE=/health \
    -e AIP_PREDICT_ROUTE=/predict \
    -e AIP_STORAGE_URI={MODEL_DIR} \
    {DEPLOY_IMAGE}
!docker container ls
!sleep 10

- To send the container's server a health check, run the following command:

In [ ]:
!curl http://localhost:7080/health

If successful, the server returns the following response:

```
{
  "status": "healthy"
}
```

- To send the container's server a prediction request, run the following commands:

In [ ]:
!curl -X POST \
  -d @serve/instances.json \
  -H "Content-Type: application/json; charset=utf-8" \
  http://localhost:7080/predict

This request uses a test sentence. If successful, the server returns prediction in below format:

```
{"predictions":["versicolor","setosa"]}
```

- To stop the container, run the following command:

In [ ]:
!docker stop local-iris

#### Deploying the serving container to Vertex Predictions
We create a model resource on Vertex AI and deploy the model to a Vertex Endpoints. You must deploy a model to an endpoint before using the model. The deployed model runs the custom container image to serve predictions.

#### Push the serving container to Container Registry
Push your container image with inference code and dependencies to your Container Registry

In [ ]:
!docker push $DEPLOY_IMAGE

## Upload the model

Next, upload your model to a `Model` resource using `Model.upload()` method, with the following parameters:

- `display_name`: The human readable name for the `Model` resource.
- `artifact`: The Cloud Storage location of the trained model artifacts.
- `serving_container_image_uri`: The serving container image.
- `serving_container_predict_route`:  HTTP path to send prediction requests to the container.
- `serving_container_health_route`: HTTP path to send health check requests to the container.
- `serving_container_ports`: The ports exposed by the container to listen to requests.
- `sync`: Whether to execute the upload asynchronously or synchronously.

If the `upload()` method is run asynchronously, you can subsequently block until completion with the `wait()` method.

In [ ]:
model_display_name = f"{APP_NAME}-{TIMESTAMP}"
model_description = "LightGBM based iris flower classifier with custom container"

MODEL_NAME = APP_NAME
health_route = "/ping"
predict_route = "/predict"
serving_container_ports = [7080]

In [ ]:
model = aiplatform.Model.upload(
    display_name=model_display_name,
    description=model_description,
    serving_container_image_uri=DEPLOY_IMAGE,
    serving_container_predict_route=predict_route,
    serving_container_health_route=health_route,
    serving_container_ports=serving_container_ports,
)

model.wait()

print(model.display_name)
print(model.resource_name)

*Example output:*

    INFO:google.cloud.aiplatform.models:Creating Model
    INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/759209241365/locations/us-central1/models/925164267982815232/operations/3458372263047331840
    INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/759209241365/locations/us-central1/models/925164267982815232
    INFO:google.cloud.aiplatform.models:To use this Model in another session:
    INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/759209241365/locations/us-central1/models/925164267982815232')

## Make batch predictions

Documentation - [Batch prediction requests - Vertex AI](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

### Make test items

You will use synthetic data as a test data items. Don't be concerned that we are using synthetic data -- we just want to demonstrate how to make a prediction.

In [ ]:
INSTANCES = [[6.7, 3.1, 4.7, 1.5], [4.6, 3.1, 1.5, 0.2]]

### Make the batch input file

Now make a batch input file, which you will store in your local Cloud Storage bucket.  Each instance in the prediction request is a list of the form:

                        [ [ content_1], [content_2] ]

- `content`: The feature values of the test item as a list.

In [ ]:
[json.dumps(record) for record in INSTANCES]

In [ ]:
import tensorflow as tf

gcs_input_uri = f"{BUCKET_NAME}/{APP_NAME}/test/batch_input/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    for i in INSTANCES:
        f.write(str(i) + "\n")

! gsutil cat $gcs_input_uri

### Make the batch prediction request

Now that your Model resource is trained, you can make a batch prediction by invoking the batch_predict() method, with the following parameters:

- `job_display_name`: The human readable name for the batch prediction job.
- `gcs_source`: A list of one or more batch request input files.
- `gcs_destination_prefix`: The Cloud Storage location for storing the batch prediction resuls.
- `instances_format`: The format for the input instances, either 'csv' or 'jsonl'. Defaults to 'jsonl'.
- `predictions_format`: The format for the output predictions, either 'csv' or 'jsonl'. Defaults to 'jsonl'.
- `machine_type`: The type of machine to use for training.
- `sync`: If set to True, the call will block while waiting for the asynchronous batch job to complete.

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

batch_predict_job = model.batch_predict(
    job_display_name=f"{APP_NAME}_TIMESTAMP",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=f"{BUCKET_NAME}/{APP_NAME}/test/batch_output/",
    instances_format="jsonl",
    predictions_format="jsonl",
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=False,
)

print(batch_predict_job)

*Example output:*

    INFO:google.cloud.aiplatform.jobs:Creating BatchPredictionJob
    <google.cloud.aiplatform.jobs.BatchPredictionJob object at 0x7f806a6112d0> is waiting for upstream dependencies to complete.
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob created. Resource name: projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296
    INFO:google.cloud.aiplatform.jobs:To use this BatchPredictionJob in another session:
    INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296')
    INFO:google.cloud.aiplatform.jobs:View Batch Prediction Job:
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5110965452507447296?project=759209241365
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296 current state:
    JobState.JOB_STATE_RUNNING

### Wait for completion of batch prediction job

Next, wait for the batch job to complete. Alternatively, one can set the parameter `sync` to `True` in the `batch_predict()` method to block until the batch prediction job is completed.

In [ ]:
batch_predict_job.wait()

*Example Output:*

    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob created. Resource name: projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328
    INFO:google.cloud.aiplatform.jobs:To use this BatchPredictionJob in another session:
    INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328')
    INFO:google.cloud.aiplatform.jobs:View Batch Prediction Job:
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/181835033978339328?project=759209241365
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 current state:
    JobState.JOB_STATE_SUCCEEDED
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob run completed. Resource name: projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328

### Get the predictions

Next, get the results from the completed batch prediction job.

The results are written to the Cloud Storage output bucket you specified in the batch prediction request. You call the method iter_outputs() to get a list of each Cloud Storage file generated with the results. Each file contains one or more prediction requests in a JSON format:

- `instance`: The prediction request.
- `prediction`: The prediction response.

In [ ]:
import json

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

*Example Output:*

    {'instance': [6.7, 3.1, 4.7, 1.5], 'prediction': 'versicolor'}

## Make online predictions

Documentation - [Online prediction request](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

## Deploy the model

Next, deploy your model for online prediction. To deploy the model, you invoke the `deploy` method, with the following parameters:

- `deployed_model_display_name`: A human readable name for the deployed model.
- `traffic_split`: Percent of traffic at the endpoint that goes to this model, which is specified as a dictionary of one or more key/value pairs.
If only one model, then specify as { "0": 100 }, where "0" refers to this model being uploaded and 100 means 100% of the traffic.
If there are existing models on the endpoint, for which the traffic will be split, then use model_id to specify as { "0": percent, model_id: percent, ... }, where model_id is the model id of an existing model to the deployed endpoint. The percents must add up to 100.
- `machine_type`: The type of machine to use for training.
- `starting_replica_count`: The number of compute instances to initially provision.
- `max_replica_count`: The maximum number of compute instances to scale to. In this tutorial, only one instance is provisioned.

In [ ]:
DEPLOYED_NAME = f"{APP_NAME}-{TIMESTAMP}"

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

*Example output:*

    INFO:google.cloud.aiplatform.models:Creating Endpoint
    INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/4087251132693348352
    INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
    INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/759209241365/locations/us-central1/endpoints/4867177336350441472')
    INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/1691336130932244480
    INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/759209241365/locations/us-central1/endpoints/4867177336350441472

### [predictions.online-prediction-automl](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl)

### Make test item

You will use synthetic data as a test data item. Don't be concerned that we are using synthetic data -- we just want to demonstrate how to make a prediction.

In [ ]:
INSTANCE = [1.4, 1.3, 5.1, 2.8]

### Make the prediction

Now that your `Model` resource is deployed to an `Endpoint` resource, you can do online predictions by sending prediction requests to the `Endpoint` resource.

#### Request

The format of each instance is:

    [feature_list]

Since the predict() method can take multiple items (instances), send your single test item as a list of one test item.

#### Response

The response from the predict() call is a Python dictionary with the following entries:

- `ids`: The internal assigned unique identifiers for each prediction request.
- `predictions`: The predicted confidence, between 0 and 1, per class label.
- `deployed_model_id`: The Vertex AI identifier for the deployed `Model` resource which did the predictions.

In [ ]:
instances_list = [INSTANCE]

prediction = endpoint.predict(instances_list)
print(prediction)

## Undeploy the model

When you are done doing predictions, you undeploy the model from the `Endpoint` resouce. This deprovisions all compute resources and ends billing for the deployed model.

In [ ]:
endpoint.undeploy_all()

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- Pipeline
- Model
- Endpoint
- AutoML Training Job
- Batch Job
- Custom Job
- Hyperparameter Tuning Job
- Cloud Storage Bucket

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME